In [1]:
using Gen

Let's dig into what exact inference looks like for multi-object tracking. 

Let's imagine a very simple model -- between 1-4 objects spawn on a 5x5 grid. What's the initial probability of a starting condition?

The probability of $N$ objects being sampled is $\frac{1}{4}$. For any given object, the probability it appears in any specific location on the grid is proportional to the scene size. Since we sample both $x$ and $y$ coordinates independently that probability is $\frac{1}{25}$. 

Now, for a scene with 4 objects, the joint probability of sampling 4 objects at 4 specific locations is: $(\frac{1}{4})(\frac{1}{25})^4$, or $6e-7$. A pretty low likelihood for any given configuration. 

We can illustrate that below with Gen:

In [124]:
@gen function sample_position(scene_size::Int64)
    """
    Return a 
    """
    x = {:x} ~ uniform_discrete(1, scene_size)
    y = {:y} ~ uniform_discrete(1, scene_size)
    return x, y
end

@gen function init_state(max_objects::Int64, scene_size::Int64)
    """
    Generative model for initial positions of an unknown number of objects in a scene
    """
    n = {:n} ~ uniform_discrete(1, max_objects)
    get_positions = Map(sample_position)
    sizes = [scene_size for i in 1:n]
    positions ~ get_positions(sizes)
    return positions
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Int64], false, Union{Nothing, Some{Any}}[nothing, nothing], var"##init_state#350", Bool[0, 0], false)

In [141]:
tr = simulate(init_state, (4, 5))
choices = get_choices(tr)
println(choices[:positions => 1 => :y])
println(exp(get_score(tr)))

3
0.0004000000000000001


In [166]:
tr, w = generate(init_state, (4, 5), choicemap((:positions => 4 => :y, 10)))
println(w)

tr, w = generate(init_state, (4, 5), choicemap((:n, 4)))
println(exp(w))

0.0
0.25


Tracking fireflies? What's next?

Rewrite exact inference in a latent-observation model? Re-do SIR? Hm...

In [173]:
firing = poisson(4)
println(firing)

5


In [180]:
using Gen
using Plots
using Distributions

# Define the firefly model
@gen function firefly_model(T::Int, N::Int)
    num_fireflies = @trace(uniform_discrete(3, 7), :num_fireflies)
    
    positions = [(@trace(uniform_discrete(1, N), (:initial_x, i)), 
                  @trace(uniform_discrete(1, N), (:initial_y, i))) 
                 for i in 1:num_fireflies]
    frequencies = [@trace(uniform(0.5, 2.0), (:frequency, i)) for i in 1:num_fireflies]
    
    trajectories = []
    glow_states = []
    
    for i in 1:num_fireflies
        trajectory = [positions[i]]
        glow_state = zeros(T)
        
        for t in 2:T
            last_x, last_y = trajectory[end]
            new_x = mod1(last_x + @trace(uniform_discrete(-1, 1), (:step_x, i, t)), N)
            new_y = mod1(last_y + @trace(uniform_discrete(-1, 1), (:step_y, i, t)), N)
            push!(trajectory, (new_x, new_y))
            
            # Update glow state with smooth ramp up and down
            cycle_position = mod(t, round(Int, 1 / frequencies[i]))
            glow_window = round(Int, 1 / frequencies[i] / 4)
            if cycle_position < glow_window
                glow_state[t] = sin(π * cycle_position / glow_window)
            elseif cycle_position < 2 * glow_window
                glow_state[t] = sin(π * (2 * glow_window - cycle_position) / glow_window)
            end
        end
        
        push!(trajectories, trajectory)
        push!(glow_states, glow_state)
    end
    
    return (trajectories, glow_states)
end

# Run the model
T = 200  # Total time steps
N = 20   # Grid size (NxN)
tr = Gen.simulate(firefly_model, (T, N))
(trajectories, glow_states) = get_retval(tr)


# Create animation
anim = @animate for t in 1:T
    p = plot(legend=false, size=(600, 600), xlabel="X Position", ylabel="Y Position", 
             title="Firefly Simulation (t=$t)", xlim=(0, N), ylim=(0, N),
             aspect_ratio=:equal, xticks=0:N, yticks=0:N, grid=true)
    
    for (i, (trajectory, glow_state)) in enumerate(zip(trajectories, glow_states))
        x, y = trajectory[t]
        glow = glow_state[t]
        
        # Plot firefly with glow effect
        scatter!(p, [x], [y], markersize=6 + 4 * glow, color=:yellow, 
                 alpha=0.3 + 0.7 * glow, markerstrokewidth=0)
        
        # Plot tail
        tail_length = 10
        start = max(1, t - tail_length)
        x_tail = [traj[1] for traj in trajectory[start:t]]
        y_tail = [traj[2] for traj in trajectory[start:t]]
        plot!(p, x_tail, y_tail, linewidth=1, alpha=0.5, color=:gray)
    end
end

# Save the animation as a GIF
gif(anim, "fireflies.gif", fps = 15)
println("Animation saved as fireflies.gif")

Animation saved as fireflies.gif


[ Info: Saved animation to /Users/yonifriedman/Research/ProbComp/Fireflies/fireflies.gif
